# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Neural Network Design: Learning from Neural Ar...,Talk,talk-neural-network-design,SSCI 2020,2020-12-02,"Canberra, Australia",https://youtu.be/DnP05lbE7dM,Neural Architecture Search (NAS) aims to optim...
1,An Incremental Algorithm for Repairing Trainin...,Talk,talk-iari-ipmu,IPMU 2016,2016-11-02,"Eindhoven, The Netherlands",https://link.springer.com/chapter/10.1007/978-...,Real-life datasets that occur in domains such ...
2,Analysis and Visualization of Missing Value Pa...,Talk,talk-misvis-ipmu,IPMU 2016,2016-11-02,"Eindhoven, The Netherlands",https://link.springer.com/chapter/10.1007/978-...,Missing values in datasets form a very relevan...
3,Local Subspace-Based Outlier Detection using G...,Talk,talk-gloss-bigdata,IEEE Big Data,2016-12-07,"Washington DC, USA",https://ieeexplore.ieee.org/document/7840717,Outlier detection in high-dimensional data is ...
4,Fuzzy Clustering for Optimally Weighted Cluste...,Talk,talk-owck-wcci,WCCI 2016,2016-07-25,"Vancouver, Canada",https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,Kriging or Gaussian Process Regression has bee...
5,From Data to Knowledge to Optimization,Invited talk,invited-phd,PhD Colloquium,2017-02-02,"Leiden, The Nethehrlands",NaN,Invited talk for thhe PhH Colloquium with Math...
6,A Novel Uncertainty Quantification Method for ...,Talk,talk-uq-ipmu,IPMU 2018,2018-06-06,"Cádiz, Spain",https://link.springer.com/chapter/10.1007/978-...,"For most regression models, their overall accu..."
7,Deep Steel: Optimizing networks to discover de...,Invited talk,invited-ecole,ECOLE workshop,2018-11-29,"Leiden, The Netherlands",NaN,"Invited talk for the ECOLE program, learning t..."
8,Deep Steel: Optimizing networks to discover de...,Invited talk,invited-tata,Tata Steel,2019-01-30,"IJmuiden, The Netherlands",NaN,"Invited talk at Tata Steel, about using deep l..."
9,Automatic Configuration of Deep Neural Network...,Talk,talk-cnn-ijcnn,IJCNN 2019,2019-07-02,"Budapest, Hungary",https://ieeexplore.ieee.org/abstract/document/...,Designing the architecture for an artificial n...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2012-03-01-talk-1.md	  2014-02-01-talk-2.md
2013-03-01-tutorial-1.md  2014-03-01-talk-3.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
